# Buckets notifications configuration

# Preparatory steps

### Please update the below variable values to reflect the actual ones used in your ecosystem

In [1]:
# Enter the main namespace name of this lab and the name of the kafka cluster name.
namespace = 'xraylab'
kafka_cluster_name = 'xraylab'

# Enter you bucket base name. This is the bucket 
bucket_base_name = 'ml-pneumonia'

# Enter you Access and Secret keys. They are the ones that were displayed in the instructions and should look similar to the below ones.
aws_access_key_id = '3XWIR321K1ERLV4FXK5D'
aws_secret_access_key = 'Fg9c9MRoKzZ41sad7xGurblqmYe0XaKtrD2ZL0ve'
region_name = 'default' #default region for the profile e.g., us-east-2

# To reduce external traffic one can use the internal cluster service endpoint - it should look something like below and can be obtained from the openshift-storage namespace
# endpoint_url = 'http://rook-ceph-rgw-ocs-storagecluster-cephobjectstore.openshift-storage.svc.cluster.local'

# The external rados GW endpoint can be obtained from the networking-routes section of the administrator view having openshift-storage selected as project.
endpoint_url = 'https://rgw-openshift-storage.apps.cluster-lv628.lv628.sandbox1664.opentlc.com/'


## Imports
Of course we'll need some libraries to work with, so import them by running the following cell.

In [2]:
import boto3
import json
import botocore
import argparse

## S3 and SNS connections
Boto3 is a standard library to interact with cloud services like S3 and SNS. As Ceph is compatible with S3 and SNS, we can directly use the library to work with the storage. First, let's create the clients to connect to the storage (you can see we are using some parameters we defined earlier).

In [3]:
s3 = boto3.client('s3',
                endpoint_url = endpoint_url,
                aws_access_key_id = aws_access_key_id,
                aws_secret_access_key = aws_secret_access_key,
                region_name = 'default',
                config=botocore.client.Config(signature_version = 's3'))

sns = boto3.client('sns', 
                endpoint_url = endpoint_url, 
                aws_access_key_id = aws_access_key_id,
                aws_secret_access_key= aws_secret_access_key,
                region_name='default', 
                config=botocore.client.Config(signature_version = 's3'))

### Check there are buckets
If the response does not list any buckets, please execute the s3-buckets notebook to create the necessary buckets and set the correct ACL.

In [4]:
for bucket in s3.list_buckets()['Buckets']:
    print(bucket['Name'])

ml-pneumonia
ml-pneumonia-datasource
ml-pneumonia-processed
ml-pneumonia-train-test-valid


# Bucket Notifications configuration

### Define the notification endpoint with the help of an array.

In [5]:
attributes = {}
attributes['push-endpoint'] = 'kafka://'+ kafka_cluster_name +'-kafka-bootstrap.'+namespace+':9092'
attributes['kafka-ack-level'] = 'broker'

### Define a function that will create a topic with those attributes (I know we will create only one topic, so a function may seem too much, but now you have a reusable snippet for when you have lots to create).

In [6]:
def create_topic(topic):
    topic_arn = sns.create_topic(Name=topic, Attributes=attributes)['TopicArn']
    return topic_arn

### Create the notification topic

In [7]:
create_topic('xray-images')

'arn:aws:sns:s3a::xray-images'

### A quick check that it has been created.

In [8]:
sns.list_topics()

{'Topics': [{'TopicArn': 'arn:aws:sns:s3a::xray-images'}],
 'ResponseMetadata': {'RequestId': '6c1dfce0-2065-47c7-8d2e-7d95b37b38cb.14449.24713',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-request-id': 'tx000000000000000006089-006185904a-3871-s3a',
   'content-type': 'application/xml',
   'content-length': '931',
   'date': 'Fri, 05 Nov 2021 20:12:58 GMT',
   'set-cookie': '9a65de55c65e51452840d54620db26a7=8046f8b0e40183853b3729d1bfae9949; path=/; HttpOnly'},
  'RetryAttempts': 0}}

### Define a notification configuration, i.e. when our topic should be used. Here it's whenever a new object is being created ("Events": \["s3:ObjectCreated:*"\]), in which case we use our topic, refering to it through its ARN (unique id, 'arn:aws:sns:s3a::xray-images'). And we apply this configuration to our base bucket, the one where the images will arrive: 

In [9]:
bucket_notifications_configuration = {
            "TopicConfigurations": [
                {
                    "Id": 'xray-images',
                    "TopicArn": 'arn:aws:sns:s3a::xray-images',
                    "Events": ["s3:ObjectCreated:*"]
                }
            ]
        }

s3.put_bucket_notification_configuration(Bucket = bucket_base_name,
        NotificationConfiguration=bucket_notifications_configuration)

{'ResponseMetadata': {'RequestId': 'tx000000000000000006091-0061859055-3871-s3a',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-request-id': 'tx000000000000000006091-0061859055-3871-s3a',
   'content-length': '0',
   'date': 'Fri, 05 Nov 2021 20:13:09 GMT',
   'set-cookie': '9a65de55c65e51452840d54620db26a7=8046f8b0e40183853b3729d1bfae9949; path=/; HttpOnly'},
  'RetryAttempts': 0}}

### Verfication that the configuration has been applied to our bucket.

In [10]:
s3.get_bucket_notification_configuration(Bucket = bucket_base_name)

{'ResponseMetadata': {'RequestId': 'tx000000000000000006093-006185905a-3871-s3a',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-request-id': 'tx000000000000000006093-006185905a-3871-s3a',
   'content-type': 'application/xml',
   'content-length': '192',
   'date': 'Fri, 05 Nov 2021 20:13:14 GMT',
   'set-cookie': '9a65de55c65e51452840d54620db26a7=8046f8b0e40183853b3729d1bfae9949; path=/; HttpOnly',
   'cache-control': 'private'},
  'RetryAttempts': 0},
 'TopicConfigurations': [{'Id': 'xray-images',
   'TopicArn': 'arn:aws:sns:s3a::xray-images',
   'Events': ['s3:ObjectCreated:*']}]}

# You're done!
Buckets have been created, notifications have been configured. You're now ready to run the demo. You can leave the notebook opened or close the tab, and go back to the Bookbag for the instructions on how to run the demo.